# Boilerplate functions

In [1]:
import cv2

In [2]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [6]:


# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))



[[22. 28.]
 [49. 64.]]


In [7]:
from scipy.io import loadmat
import numpy as np
import re
import numpy as np
import time
import math
import pandas as pd
import os

import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.ticker import MaxNLocator

import tensorflow as tf
import cv2

from sklearn.model_selection import train_test_split
from keras.preprocessing import image

from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from glob import glob
from tqdm import tqdm

import pickle
%matplotlib inline

## Function to read the video frame by frame and store a few frames

In [ ]:
def frame_capture(filename):
    cap = cv2.VideoCapture(filename)
    count = 0
    frame_prefix = filename.split('/')[6]+"_"\
                  +filename.split('/')[7]+"_"\
                  +filename.split('/')[8]+"_"\
                  +(filename.split('/')[9]).split('.')[0]
    
    frameRate = cap.get(5)
    while(cap.isOpened()):
        frameId = cap.get(1)
        ret, frame = cap.read()

        if ret == True:
            # Display the resulting frame : cv2.imshow('Frame',frame)
            
            if (frameId % math.floor(frameRate) == 0):
                framename = '/media/mynewdrive/data/Retail_AI/3DPeS/'+frame_prefix+"_frame%d.jpg" % count
                if not cv2.imwrite(framename, frame):
                    raise Exception("Could not write image")

            # Press Q on keyboard to  exit
            count+=1
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        # Break the loop
        else: 
            break

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()

In [ ]:
import os
my_dir = '/media/mynewdrive/data/Retail_AI/3DPeS/Video_Set_1/'
for dir_2 in os.listdir(my_dir):
    dir_3 = os.path.join(my_dir,dir_2)
    for dir_4 in os.listdir(dir_3):
        dir_5 = os.path.join(dir_3,dir_4)
        for dir_6 in os.listdir(dir_5):
            filename = os.path.join(dir_5,dir_6)
            frame_capture(filename)                  

# Load YoloV3

In [8]:
#load yolo

net = cv2.dnn.readNet("yolov3.weights.1","darknet/cfg/yolov3.cfg")
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))
    

In [7]:
# # Lets try counting all file names
# dir_name = 'Video_Set_1_Frames'
# count = 0
# for frames in os.listdir(dir_name):
#     count += 1
# count
    

2708

In [30]:
def read_detect_show (img_name):
    img = cv2.imread(img_name)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    
    #detect objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416),(0,0,0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                #Object detected
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*width)
                w = int(detection[2] * width)
                h = int(detection[3] * width)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 1, color, 3)
    yolo_img_name = img_name.split('/')[0]+"_YOLO/"+(img_name.split('/')[1]).split('.')[0]+"_yolo.jpg"
    cv2.imwrite(yolo_img_name,img)
#     cv2.imshow("Image", img)
#     cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
dir_name = 'Video_Set_1_Frames'
count = 0
for frames in os.listdir(dir_name):
    count += 1
    print(count)
    read_detect_show(dir_name+"/"+frames)
    